# Milestone P3

## Abstract

## Part 1: Data familiarization

### Part 1.1: Loading data

### Part 1.2: Data scraping

### Part 1.3: Role identification

## Part 2: Data preprocessing

### Part 2.1: Adapt to inflation

### Part 2.2: Clear absurdities

### Part 2.3: Format languages/countries/genres/ethnicities

### Part 2.4: One-hot encoding of categorical features

## Part 3: Features study

### Part 3.1: Gender study

### Part 3.2: Space study

### Part 3.3: Time study

### Part 3.4: Ethnicity study

## Part 4: Modelling

### Part 4.1: Input analysis

### Part 4.2: Hyperparameter tuning

### Part 4.3: Model training and testing

### Part 3.4: Model utilization (fun part)